In [14]:
from SPARQLWrapper import SPARQLWrapper, JSON
import rdflib
# sparql = SPARQLWrapper("http://data.judaicalink.org/sparql/query")

sparqlquery = SPARQLWrapper("http://localhost:3030/tutorial/query")
sparqlupdate = SPARQLWrapper("http://localhost:3030/tutorial/update")

In [15]:
prefixes = [
    ("rdf", "http://www.w3.org/1999/02/22-rdf-syntax-ns#"),
    ("rdfs", "http://www.w3.org/2000/01/rdf-schema#"),
    ("owl", "http://www.w3.org/2002/07/owl#"),
    ("skos", "http://www.w3.org/2004/02/skos/core#"),
    ("foaf", "http://xmlns.com/foaf/0.1/"),
    ("dc", "http://purl.org/dc/terms/"),
    ("h", "http://"),
]

INT = "http://www.w3.org/2001/XMLSchema#integer"
a = "rdf:type"

In [34]:
def literal(value, langOrType=None):
    res = '"{}"'.format(value)
    if langOrType==None:
        return res
    if len(langOrType)==2:
        return "{}@{}".format(res, langOrType)
    else:
        return "{}^^{}".format(res, langOrType)


def expand(uri, prefixes=prefixes):
    if "<" == uri[0]:
        return uri
    if '"' == uri[0] or "'" == uri[0]:
        return uri
    if "http://" == uri[:7] or "https://" == uri[:8]:
        return "<{}>".format(uri)
    for p in prefixes:
        if "{}:".format(p[0]) == uri[:len(p[0])+1]:
            return "<{}>".format(uri.replace("{}:".format(p[0]), p[1]))
    return '"{}"'.format(uri)


def insert(data, prefixes=prefixes):
    prefixstring = "\n".join(map(lambda x: "PREFIX {}: <{}>".format(x[0], x[1]), prefixes))

    query = "%s INSERT { %s } WHERE {}" % (prefixstring, data)
    sparqlupdate.setQuery(query)
    sparqlupdate.setReturnFormat(JSON)
    sparqlupdate.method = 'POST'
    return sparqlupdate.query()

def select(variables, where, prefixes, limit=10):
    prefixstring = "\n".join(map(lambda x: "PREFIX {}: <{}>".format(x[0], x[1]), prefixes))
    query = "%s\n\nSELECT %s WHERE { %s } LIMIT %s" % (prefixstring, variables, where, limit)
    sparqlquery.setQuery(query)
    sparqlquery.setReturnFormat(JSON)
    sparqlquery.method = 'GET'
    return sparqlquery.query().convert()

def getValues(result, *variable):
    for b in result["results"]["bindings"]:
        res = []
        for v in variable:
            res.append(b[v[1:]]["value"])
        yield tuple(res)

def get(uri):
    uri = expand(uri)
    uri = uri[1:-1]
    graph = rdflib.Graph()
    graph.parse(uri)
    return graph.serialize(format='nt').decode()


def triple(subject, predicate, object, prefixes=prefixes):
    return "{} {} {} .".format(expand(subject, prefixes),
                             expand(predicate, prefixes),
                             expand(object, prefixes))

In [35]:
neuedaten = get("http://dbpedia.org/resource/Oskar_Trautmann")

In [ ]:
insert(neuedaten, prefixes)

# insert("""<http://example.org/data/5> dc:creator h:Kai2 """)
# insert(triple("h:example.org/data/4", "dc:title", literal("Cool Stuff", "en")))
# insert(triple("h:example.org/data/4", a, "skos:Concept"))
# print(triple("h:example.org/data/4", "dc:title", literal("Cool Stuff", "en")))
# print(expand("dc:creator"))
# print(expand("Hello World"))
# print(expand(literal("Hello World", "en")))
# print(expand("http://www.example.org"))



# r = select("?s ?p ?o", "?s ?p ?o", prefixes, 30)
# for s, p, o in getValues(r, "?s", "?p", "?o"):
#    print(s, p, o)

In [27]:
g = rdflib.Graph()
g.parse('http://mediagraph.link/vndb/characters/33')

<Graph identifier=N500e76943721425cbc20740a16558d6b (<class 'rdflib.graph.Graph'>)>

In [30]:
for triple in g:
    print(triple)

(rdflib.term.URIRef('http://mediagraph.link/vndb/characters/33'), rdflib.term.URIRef('http://mediagraph.link/ont/vndb/characterTrait'), rdflib.term.Literal('Boots (Shoes)'))
(rdflib.term.URIRef('http://mediagraph.link/vndb/characters/33'), rdflib.term.URIRef('http://mediagraph.link/ont/vndb/characterTrait'), rdflib.term.Literal('Other Route Death (Avoidable Death)'))
(rdflib.term.URIRef('http://mediagraph.link/vndb/characters/33'), rdflib.term.URIRef('http://mediagraph.link/ont/vndb/bustSize'), rdflib.term.Literal('73'))
(rdflib.term.URIRef('http://mediagraph.link/vndb/characters/33'), rdflib.term.URIRef('http://mediagraph.link/ont/vndb/characterTrait'), rdflib.term.Literal('Polyamory (Open Relationship)'))
(rdflib.term.URIRef('http://mediagraph.link/vndb/characters/33'), rdflib.term.URIRef('http://mediagraph.link/ont/vndb/characterTrait'), rdflib.term.Literal('Teen (Apparent Age)'))
(rdflib.term.URIRef('http://mediagraph.link/vndb/characters/33'), rdflib.term.URIRef('http://mediagraph